In [1]:
import numpy as np 
import pandas as pd 
import json
import bq_helper
from pandas.io.json import json_normalize
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.metrics import mean_squared_error
import functools
from multiprocessing import Pool
import logging
import gc
import logging
import matplotlib.pyplot as plt
import time
from sklearn.metrics import mean_squared_error
from scipy.stats import stats
from ast import literal_eval

In [2]:
def parse(csv_path='data/train_v2.csv', nrows=None):

    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    df['hits']=df['hits'].apply(literal_eval)
    df['hits']=df['hits'].str[0]
    df=df[pd.notnull(df['hits'])]
    
    df['customDimensions']=df['customDimensions'].apply(literal_eval)
    df['customDimensions']=df['customDimensions'].str[0]
    df=df[pd.notnull(df['customDimensions'])]
    
    
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource','hits','customDimensions']
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    return df
    
def load_data(nrows=None):
    train_df = parse(nrows=nrows)
    test_df = parse("data/test_v2.csv",nrows)
    return train_df,test_df

In [3]:
train_df,test_df=load_data()

In [5]:
train_df.to_csv('train_df.csv')
test_df.to_csv('test_df.csv')